In [1]:
import sys
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, LongType
from delta.pip_utils import configure_spark_with_delta_pip

In [2]:
def get_spark_builder():
    builder = SparkSession.builder \
        .master('local[1]') \
        .appName('first_script.main') \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config('spark.databricks.delta.schema.autoMerge.enabled', True) \
        .config('spark.hadoop.hive.metastore.client.factory.class', 'com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory') \

    return builder

def get_spark_session(mode='prod'):
    builder = get_spark_builder()

    if mode == 'prod':
        spark = builder.getOrCreate()
    else:
        builder = builder.config("spark.sql.warehouse.dir", mode)
        spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()
    return spark

In [3]:
spark = get_spark_session('test')

23/10/01 17:13:11 WARN Utils: Your hostname, spiriel resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface enp3s0)
23/10/01 17:13:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ahow/virtual_envs/ahow/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ahow/.ivy2/cache
The jars for the packages stored in: /home/ahow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1c980090-47b5-4536-83d9-3559feeb1082;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 128ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [4]:
df = spark.read.format('delta').load(
    '/home/ahow/MyGitHub/local_bucket/delta-lake/clima_tempo/forecast_72/'
)
df.persist()

23/10/01 17:13:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[date: timestamp, date_br: string, humidity: double, pressure: double, rain: struct<precipitation:double>, temperature: bigint, wind: struct<direction:string,directiondegrees:double,gust:double,velocity:double>, precipitation: double, wind_direction: string, wind_directiondegrees: double, wind_gust: double, wind_velocity: double, country: string, id: int, state: string, filename: string, dt_extract: timestamp, dt: timestamp]

In [11]:
df.createOrReplaceTempView("temp")
spark.sql("SELECT * FROM temp").show()

+-------------------+-------------------+--------+--------+-----+-----------+--------------------+-------------+--------------+---------------------+---------+-------------+-------+----+-----+--------------------+--------------------+-------------------+
|               date|            date_br|humidity|pressure| rain|temperature|                wind|precipitation|wind_direction|wind_directiondegrees|wind_gust|wind_velocity|country|  id|state|            filename|          dt_extract|                 dt|
+-------------------+-------------------+--------+--------+-----+-----------+--------------------+-------------+--------------+---------------------+---------+-------------+-------+----+-----+--------------------+--------------------+-------------------+
|2023-09-28 21:00:00|29/09/2023 00:00:00|    89.8|  1018.4|{0.0}|         17|{ESE, 129.3, 31.7...|          0.0|           ESE|                129.3|     31.7|         15.7|     BR|3477|   SP|s3://ahow-delta-l...|2023-09-27 01:39:...|2

In [ ]:
spark.sql(f"""

""")